
# Objetivo 2: Generación de modelos

Pipeline completo para la predicción del crecimiento de datos de biodiversidad en GBIF.

Este script implementa el flujo de trabajo de principio a fin para modelar datos de panel
de series temporales, incluyendo:
1.  Carga datos de PA_dataAnalysis y preparación
2.  Ingeniería de características temporales (lags y ventanas móviles).
3.  Un marco de validación cruzada robusto para series de tiempo (ventana expansiva).
4.  Preprocesamiento (imputación y escalado) dentro del bucle de validación para evitar fuga de datos.
5.  Entrenamiento y evaluación comparativa de cuatro modelos:
    - Prophet.
    - Random Forest.
    - XGBoost.
    - Red Neuronal LSTM (para modelado secuencial).
6.  Selección del mejor modelo basado en métricas de rendimiento (MAE, RMSE, R²).
7.  Reentrenamiento del modelo final y generación de pronósticos para Colombia hasta 2030
    bajo dos escenarios de políticas.

In [57]:
# =============================================================================
# 1. IMPORTACIÓN DE LIBRERÍAS Y CONFIGURACIÓN INICIAL
# =============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm import tqdm

# Preprocesamiento y modelado de Scikit-Learn
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Modelos especializados

import xgboost as xgb
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet

# Configuraciones generales
warnings.filterwarnings('ignore')
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (15, 7)

In [58]:
# Modelado de Deep Learning con TensorFlow/Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [59]:
#Carga de datos

url = "https://raw.githubusercontent.com/rortizgeo/Maestria_CD_Proyecto-Aplicado/main/Data_final.csv"
Data_final = pd.read_csv(url)

# Eliminación de columnas por tener muchos vacíos y no ser posible completarlas con imputación. (pensar en otras estrategias)
columns_to_drop = ['Overall score', 'areas_protegidas']
Data_final = Data_final.drop(columns=columns_to_drop)

Para aplicar modelos como Random Forest y XGBoost, es necesario agregar características de temporalidad en los datos, para lo cuál es necesario calcular retardos, que se deben aplicar teniendo en cuenta un análisis del ACF Y PACF, así como la incorporación de los tiempos del retardo como hiperparámetros. 

Los modelos basados en árboles como Random Forest y XGBoost no son conscientes de la secuencia temporal de los datos y no pueden "extrapolar" tendencias más allá de los valores que han visto en el entrenamiento. Por lo tanto, es necesario convertir la información temporal en características que el modelo pueda entender. La creación de retardos (lags) y estadísticas de ventana móvil es la técnica estándar para lograrlo. Se podría identificar el número de retardos como un hiperparámetro, guiado por análisis de ACF y PACF (Ver EDA)

In [60]:
# =============================================================================
# 2. INGENIERÍA DE CARACTERÍSTICAS TEMPORALES (OPTIMIZADA)
# =============================================================================



print("\nPaso 2: Realizando ingeniería de características temporales...")

TARGET = 'occurrenceCount_publisher'

def create_temporal_features_optimized(data, features_to_lag, 
                                     lags=[1, 2, 3, 4, 5], 
                                     roll_windows=[3, 5, 7],
                                     fill_na=0):
    """
    Genera características temporales y completa automáticamente con 0
    los valores NaN generados, según la lógica del negocio.
    
    Parámetros
    ----------
    data : pd.DataFrame
        Dataset con al menos 'country' y variables numéricas.
    features_to_lag : list
        Lista de columnas numéricas a transformar.
    lags : list
        Lista de retardos.
    roll_windows : list
        Lista de ventanas móviles.
    fill_na : int/float
        Valor para completar NaN (0 por defecto según lógica de negocio).
        
    Retorna
    -------
    DataFrame con nuevas características y NaN completados.
    """
    
    df_copy = data.copy()
    
    for feature in features_to_lag:
        # Características de lag
        for lag in lags:
            lag_col = f'{feature}_lag{lag}'
            df_copy[lag_col] = df_copy.groupby('country')[feature].shift(lag)
            df_copy[lag_col] = df_copy[lag_col].fillna(fill_na)
        
        # Características de ventana móvil
        for w in roll_windows:
            # Rolling mean
            mean_col = f'{feature}_rollmean{w}'
            df_copy[mean_col] = (
                df_copy.groupby('country')[feature]
                .shift(1)
                .rolling(window=w, min_periods=1)
                .mean()
            )
            df_copy[mean_col] = df_copy[mean_col].fillna(fill_na)
            
            # Rolling std
            std_col = f'{feature}_rollstd{w}'
            df_copy[std_col] = (
                df_copy.groupby('country')[feature]
                .shift(1)
                .rolling(window=w, min_periods=1)
                .std()
            )
            df_copy[std_col] = df_copy[std_col].fillna(fill_na)
    
    return df_copy

# ===========================
# Uso del código optimizado
# ===========================
features_to_lag = [
    "occurrenceCount_publisher", "pib_per_capita",
    "gasto_educacion_gobierno", "gasto_educacion_pib"
]

# Crear dataset con nuevas features (completando con 0)
df_featured = create_temporal_features_optimized(
    Data_final,
    features_to_lag=features_to_lag,
    lags=[1, 2, 3, 4, 5],  # Reducido para evitar overfitting
    roll_windows=[3, 5, 7],  # Reducido para evitar overfitting
    fill_na=0  # ¡IMPORTANTE! Completar con 0 según lógica de negocio
)

print("Ingeniería de características completada.")
print(f"Shape del dataset: {df_featured.shape}")
print(f"Valores NaN restantes: {df_featured.isnull().sum().sum()}")

# Mostrar estadísticas de las nuevas características
print("\nEstadísticas de las nuevas características:")
new_features = [col for col in df_featured.columns if any(x in col for x in ['_lag', '_roll'])]
print(df_featured[new_features].describe())



Paso 2: Realizando ingeniería de características temporales...
Ingeniería de características completada.
Shape del dataset: (656, 58)
Valores NaN restantes: 0

Estadísticas de las nuevas características:
       occurrenceCount_publisher_lag1  occurrenceCount_publisher_lag2  \
count                    6.560000e+02                    6.560000e+02   
mean                     1.530224e+07                    1.257584e+07   
std                      5.295431e+07                    4.383457e+07   
min                      0.000000e+00                    0.000000e+00   
25%                      1.549475e+04                    1.335000e+02   
50%                      1.176419e+06                    5.515200e+05   
75%                      9.340858e+06                    7.115654e+06   
max                      7.401771e+08                    5.946568e+08   

       occurrenceCount_publisher_lag3  occurrenceCount_publisher_lag4  \
count                    6.560000e+02                    6.56000

In [61]:
# Validar que no quedan NaN
assert df_featured.isnull().sum().sum() == 0, "¡Aún hay valores NaN!"

# Verificar distribución de las nuevas features
print("Distribución de valores en características temporales:")
for col in new_features:
    zero_percentage = (df_featured[col] == 0).mean() * 100
    print(f"{col}: {zero_percentage:.1f}% ceros")

Distribución de valores en características temporales:
occurrenceCount_publisher_lag1: 18.8% ceros
occurrenceCount_publisher_lag2: 25.0% ceros
occurrenceCount_publisher_lag3: 31.2% ceros
occurrenceCount_publisher_lag4: 37.5% ceros
occurrenceCount_publisher_lag5: 43.8% ceros
occurrenceCount_publisher_rollmean3: 9.9% ceros
occurrenceCount_publisher_rollstd3: 15.4% ceros
occurrenceCount_publisher_rollmean5: 5.9% ceros
occurrenceCount_publisher_rollstd5: 7.6% ceros
occurrenceCount_publisher_rollmean7: 3.7% ceros
occurrenceCount_publisher_rollstd7: 4.3% ceros
pib_per_capita_lag1: 6.2% ceros
pib_per_capita_lag2: 12.5% ceros
pib_per_capita_lag3: 18.8% ceros
pib_per_capita_lag4: 25.0% ceros
pib_per_capita_lag5: 31.2% ceros
pib_per_capita_rollmean3: 0.2% ceros
pib_per_capita_rollstd3: 0.3% ceros
pib_per_capita_rollmean5: 0.2% ceros
pib_per_capita_rollstd5: 0.3% ceros
pib_per_capita_rollmean7: 0.2% ceros
pib_per_capita_rollstd7: 0.3% ceros
gasto_educacion_gobierno_lag1: 6.2% ceros
gasto_educacio

In [62]:
# =============================================================================
# 3. PREPARACIÓN PARA EL MODELADO Y VALIDACIÓN
# =============================================================================
print("\nPaso 3: Preparando el marco de validación y los datos para el modelado...")

# Definir variable objetivo
TARGET = 'occurrenceCount_publisher'

# Definir variables predictoras (Decidir con Daniel cuáles usar basándose en EDA y disponibilidad. Preguntar si es posible usar todas sin complejizar el modelo)
features = [
    'PC1', 'PC2', 'pib_per_capita', 'gasto_educacion_gobierno',
    'gasto_educacion_pib', 'superficie_total_km2', "country", "region", "incomeLevel"
    f"{TARGET}_lag1", f"{TARGET}_lag2", f"{TARGET}_rollmean3", f"{TARGET}_rollstd3"
]

# Filtrar las variables que realmente existen en el DataFrame
features = [f for f in features if f in df_featured.columns]

# Definir X (predictoras) e y (objetivo)
X = df_featured[features].copy()
y = df_featured[TARGET].copy()

# Configurar validación cruzada para series de tiempo
n_splits = 5  # número de pliegues
tscv = TimeSeriesSplit(n_splits=n_splits)

# Diccionario para almacenar resultados
results = {
    'Prophet': None,
    'RandomForest': None,
    'XGBoost': None,
    'LSTM': None
}

print("Features seleccionadas:", features)
print("X shape:", X.shape)
print("y shape:", y.shape)



Paso 3: Preparando el marco de validación y los datos para el modelado...
Features seleccionadas: ['PC1', 'PC2', 'pib_per_capita', 'gasto_educacion_gobierno', 'gasto_educacion_pib', 'superficie_total_km2', 'country', 'region', 'occurrenceCount_publisher_lag2', 'occurrenceCount_publisher_rollmean3', 'occurrenceCount_publisher_rollstd3']
X shape: (656, 11)
y shape: (656,)


Pruebas de ejecución

In [63]:
# =============================================================================
# A. CREACIÓN DE SECUENCIAS Y FUNCIONES DE MODELOS
# =============================================================================

def create_lstm_sequences_global(df, features, target, look_back=3):
    """
    Crea secuencias LSTM para todos los países antes del split train/test.
    Retorna X_seq, y_seq, years, countries (alineados).
    """
    X_seq, y_seq, years, countries = [], [], [], []

    for country in df['country'].unique():
        df_country = df[df['country'] == country].sort_values('year')

        X_country = df_country[features].values.astype(np.float32)
        y_country = df_country[target].values.astype(np.float32)
        years_country = df_country['year'].values

        if len(X_country) > look_back:
            for i in range(len(X_country) - look_back):
                X_seq.append(X_country[i:(i + look_back)])
                y_seq.append(y_country[i + look_back])
                years.append(years_country[i + look_back])
                countries.append(country)

    return np.array(X_seq, dtype=np.float32), np.array(y_seq, dtype=np.float32), np.array(years), np.array(countries)


# ---- Modelos ----
def train_random_forest(X_train, y_train, X_test, y_test):
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)
    return y_test, rf.predict(X_test)

def train_xgboost(X_train, y_train, X_test, y_test):
    model = xgb.XGBRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    return y_test, model.predict(X_test)

def train_lstm(X_train, y_train, X_test, y_test, look_back):
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.float32)
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.float32)

    if X_train.shape[0] == 0 or X_test.shape[0] == 0:
        return None, None

    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(50, activation='relu', input_shape=(look_back, X_train.shape[2])),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mae')
    model.fit(X_train, y_train, epochs=10, batch_size=16, verbose=0)

    return y_test, model.predict(X_test, verbose=0).flatten()

def train_prophet(df, train_years, test_years, target):
    prophet_df = df[['year', target]].rename(columns={'year': 'ds', target: 'y'})
    prophet_train = prophet_df[prophet_df['ds'].isin(train_years)]
    prophet_test = prophet_df[prophet_df['ds'].isin(test_years)]

    m = Prophet(yearly_seasonality=True, daily_seasonality=False)
    m.fit(prophet_train)
    forecast = m.predict(prophet_test[['ds']])

    return prophet_test['y'].values, forecast['yhat'].values


In [ ]:
# =============================================================================
# 4. EJECUCIÓN DE MODELOS CON CROSS-VALIDATION
# =============================================================================
results = {'Prophet': [], 'RandomForest': [], 'XGBoost': [], 'LSTM': []}

# Features numéricas para LSTM
features_lstm = df_featured.select_dtypes(include=np.number).columns.tolist()
features_lstm = [col for col in features_lstm if col != TARGET]

look_back = 3
X_seq, y_seq, years_seq, countries_seq = create_lstm_sequences_global(
    df_featured, features_lstm, TARGET, look_back
)

unique_years = df_featured['year'].unique()
unique_years.sort()

for fold, (train_idx, test_idx) in enumerate(tscv.split(unique_years)):
    print(f"\n===== FOLD {fold+1}/{n_splits} =====")

    train_years = [unique_years[i] for i in train_idx]
    test_years = [unique_years[i] for i in test_idx]

    # === Random Forest / XGBoost ===
    mask_train = df_featured['year'].isin(train_years)
    mask_test = df_featured['year'].isin(test_years)

    X_train_tab = X.loc[mask_train].select_dtypes(include=np.number)
    y_train_tab = y.loc[mask_train]
    X_test_tab = X.loc[mask_test].select_dtypes(include=np.number)
    y_test_tab = y.loc[mask_test]

    imputer = IterativeImputer(max_iter=10, random_state=42)
    scaler = StandardScaler()

    X_train_tab = scaler.fit_transform(imputer.fit_transform(X_train_tab))
    X_test_tab = scaler.transform(imputer.transform(X_test_tab))

    y_true_rf, y_pred_rf = train_random_forest(X_train_tab, y_train_tab, X_test_tab, y_test_tab)
    results['RandomForest'].append((y_true_rf, y_pred_rf))

    y_true_xgb, y_pred_xgb = train_xgboost(X_train_tab, y_train_tab, X_test_tab, y_test_tab)
    results['XGBoost'].append((y_true_xgb, y_pred_xgb))

    # === LSTM ===
    mask_train_lstm = np.isin(years_seq, train_years)
    mask_test_lstm = np.isin(years_seq, test_years)

    X_train_lstm, y_train_lstm = X_seq[mask_train_lstm], y_seq[mask_train_lstm]
    X_test_lstm, y_test_lstm = X_seq[mask_test_lstm], y_seq[mask_test_lstm]

    y_true_lstm, y_pred_lstm = train_lstm(X_train_lstm, y_train_lstm, X_test_lstm, y_test_lstm, look_back)
    if y_true_lstm is not None:
        results['LSTM'].append((y_true_lstm, y_pred_lstm))
    else:
        results['LSTM'].append(([], []))

    # === Prophet ===
    try:
        y_true_prophet, y_pred_prophet = train_prophet(df_featured, train_years, test_years, TARGET)
        results['Prophet'].append((y_true_prophet, y_pred_prophet))
    except Exception as e:
        results['Prophet'].append(([], []))
        print(f"Prophet falló en fold {fold+1}: {e}")



===== FOLD 1/5 =====


2025-09-11 15:38:53.306873: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-09-11 15:38:53.463338: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_10.
2025-09-11 15:38:57.283884: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
15:38:57 - cmdstanpy - INFO - Chain [1] start processing
15:38:57 - cmdstanpy - INFO - Chain [1] done processing



===== FOLD 2/5 =====


2025-09-11 15:38:58.612926: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-09-11 15:38:58.761994: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_10.
2025-09-11 15:39:04.298596: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
15:39:04 - cmdstanpy - INFO - Chain [1] start processing
15:39:04 - cmdstanpy - INFO - Chain [1] done processing



===== FOLD 3/5 =====


2025-09-11 15:39:05.673273: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-09-11 15:39:05.821698: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_10.
2025-09-11 15:39:13.140203: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
15:39:13 - cmdstanpy - INFO - Chain [1] start processing
15:39:13 - cmdstanpy - INFO - Chain [1] done processing



===== FOLD 4/5 =====


2025-09-11 15:39:16.162846: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-09-11 15:39:16.321206: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_10.
2025-09-11 15:39:25.688516: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
15:39:25 - cmdstanpy - INFO - Chain [1] start processing
15:39:26 - cmdstanpy - INFO - Chain [1] done processing



===== FOLD 5/5 =====


2025-09-11 15:39:27.241992: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-09-11 15:39:27.395695: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_10.
2025-09-11 15:39:38.579410: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
15:39:38 - cmdstanpy - INFO - Chain [1] start processing
15:39:38 - cmdstanpy - INFO - Chain [1] done processing


In [ ]:
# =============================================================================
# 5. EVALUACIÓN DE MÉTRICAS
# =============================================================================
def compute_metrics(y_true, y_pred):
    if len(y_true) == 0 or len(y_pred) == 0:
        return None
    return {
        "MAE": mean_absolute_error(y_true, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_true, y_pred)),
        "R2": r2_score(y_true, y_pred),
        "MAPE": np.mean(np.abs((y_true - y_pred) / (y_true + 1e-6))) * 100  # evitar div/0
    }

summary = {}
print("\n=== Resultados finales ===")
for model, folds in results.items():
    fold_metrics = []
    for (y_true, y_pred) in folds:
        m = compute_metrics(np.array(y_true), np.array(y_pred))
        if m:
            fold_metrics.append(m)
    if fold_metrics:
        avg_metrics = {k: np.mean([fm[k] for fm in fold_metrics]) for k in fold_metrics[0]}
        summary[model] = avg_metrics
        print(f"\n{model}:")
        for k, v in avg_metrics.items():
            print(f"  {k}: {v:.4f}")
    else:
        print(f"\n{model}: sin resultados")




=== Resultados finales ===

Prophet:
  MAE: 29660934.7101
  RMSE: 69381350.3311
  R2: -0.0035
  MAPE: 36026434878106.0156

RandomForest:
  MAE: 7177462.2481
  RMSE: 27314627.1085
  R2: 0.8203
  MAPE: 253306563716.9294

XGBoost:
  MAE: 6560130.4319
  RMSE: 24589128.7395
  R2: 0.8219
  MAPE: 459248271655.9100

LSTM:
  MAE: 5699076.1000
  RMSE: 13458134.1243
  R2: 0.8699
  MAPE: 6765589545011.7100
